#### Log - Regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.miss_glm_faster import MissGLM, MissGLMSelector

# Set parameters
n = 500  # number of subjects
p = 5    # number of explanatory variables
mu_star = np.arange(0, p)/2  # mean of the explanatory variables
sd = np.arange(1, p+1)/3      # standard deviations
seed = 42

np.random.seed(seed)

# New Correlation matrix
C = np.array([
    [1,   0., 0., 0,   0],
    [0.,  1,   0., 0., 0],
    [0.,  0., 1,   0.6, 0.3],
    [0,   0., 0.6, 1,   0.5],
    [0,   0,   0.3, 0.5, 1]
])

# Covariance matrix
Sigma_star = np.diag(sd) @ C @ np.diag(sd)

# New Coefficients
beta_star = np.array([1.2, -0.5, 0.8, -1.1, 0.4])
beta0_star = 0.5  # intercept

beta_true = np.hstack([beta0_star, beta_star])
print("True coefficients: ", beta_true)

# Design matrix
X_complete = np.random.randn(n, p) @ np.linalg.cholesky(Sigma_star).T + mu_star

# Response vector
p1 = 1 / (1 + np.exp(-(X_complete @ beta_star + beta0_star)))
y = (np.random.rand(n) < p1).astype(int)

# Print summary statistics
print("Mean of each explanatory variable (X):", np.mean(X_complete, axis=0))
print("Generated response vector (y):", y[:10])  # show first 10 responses for verification


# Set the proportion of missing values
p_miss = 0.20

# Generate missing values MCAR
X_miss = X_complete.copy()
X_miss[np.random.rand(*X_miss.shape) < p_miss] = np.nan

In [ ]:

model = MissGLM(maxruns=500, tol_em=1e-7, var_cal=True, ll_obs_cal=True, seed=seed, k1=50, tau=1)

# Fit the model
model.fit(X_miss, y, save_trace=True)

In [ ]:
model.ll_obs

In [ ]:
b = np.array(model.trace["beta"])
plt.figure()

plt.plot(b[:, 0], label="Intercept")
for i in range(p):
    plt.plot(b[:, i+1], label=f"beta_{i+1}")
plt.legend()
plt.show()

In [5]:
n_preds = 300
np.random.seed(11)
newX = np.random.randn(n_preds, p) @ np.linalg.cholesky(Sigma_star).T + mu_star
newX_miss = newX.copy()
newX_miss[np.random.rand(*newX.shape) < p_miss] = np.nan

preds = model.predict_proba(newX_miss, method="impute")[:,1]

In [ ]:
# plot prediction vs true

true_prob = 1 / (1 + np.exp(-(newX @ beta_star + beta0_star)))

plt.figure()
plt.plot(true_prob, preds, 'o')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlabel("True probability")
plt.ylabel("Predicted probability")
plt.title("Prediction vs True")
plt.show()


# compute R²

R2 = 1 - np.sum((true_prob - preds)**2) / np.sum((true_prob - np.mean(true_prob))**2)
print("R²: ", R2)

In [ ]:
# plot C.I of coefficients
# with real coef in red

plt.errorbar(np.arange(p+1), model.coef_, yerr=model.std_err * 1.96, fmt='o', label='Estimated')
plt.plot(np.arange(p+1), beta_true, 'ro', label='True')
plt.legend()
plt.show()


#### Model Selection

In [8]:
X_null = np.random.randn(n, 1) + 1

# concat above X complete and X_null
X_complete_null = np.hstack([X_complete, X_null])

# missing MCAR
patterns = np.random.rand(n, X_complete_null.shape[1]) < p_miss
X_complete_null[patterns] = np.nan

In [ ]:
model_selection = MissGLMSelector()
best_model = model_selection.fit(X_complete_null, y, progress_bar=True)

In [ ]:
model_selection.get_support(indices=False)

In [ ]:
best_model.coef_

In [13]:
best_model.std_err

#### Test speed

In [ ]:
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.CUMULATIVE)
# Now you have two options, either print the data or save it as a file
stats.print_stats() # Print The Stats